### Graph Theory Hubs

In [8]:
# Packages
from nilearn.connectome import ConnectivityMeasure
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
import scipy.io
import os
import pickle

In [9]:
# Data
def filter_strings_not_in_list(strings_list, substrings_list):
    filtered_list = []
    for s in strings_list:
        if not any(substring in s for substring in substrings_list):
            filtered_list.append(s)
    return filtered_list
folder_path = '/path_to/ROISignals_FunImgARCWF'  # Pfad zum Ordner
strings_list = []
for file_name in os.listdir(folder_path):
    strings_list.append(file_name)
with open('/path_to/data_tools/first_id_list.pkl', 'rb') as file:
    data_list_substrings = pickle.load(file)

filtered_list = filter_strings_not_in_list(strings_list, data_list_substrings)
print(len(filtered_list))

1530


In [14]:
labels = []

for j, file_name in enumerate(filtered_list[:1300]):
    
    degree_list = []
    betweenness_list = []
    clustering_list = []
    closeness_list = []
    hubs_score =[]
    
    mat = scipy.io.loadmat(os.path.join(folder_path, file_name))
    matrix = mat['ROISignals']

    aal_array = matrix[:, :116]  # 1~116: Automated Anatomical Labeling (AAL) atlas (Tzourio-Mazoyer et al., 2002)
    #hoac_array = matrix[:, 116:212]   # 117~212: Harvard-Oxford atlas (Kennedy et al., 1998)– cortical areas
    #hoas_array = matrix[:, 212:228]   # 213~228: Harvard-Oxford atlas (Kennedy et al., 1998)– subcortical areas
    #ccl_array = matrix[:, 228:428]    # 229~428: Craddock’s clustering 200 ROIs (Craddock et al., 2012)
    #zrp_array = matrix[:, 428:1408]   # 429~1408: Zalesky’s random parcelations (compact version: 980 ROIs) (Zalesky et al., 2010)
    #dbf_array = matrix[:, 1408:1568]  # 1409~1568: Dosenbach’s 160 functional ROIs (Dosenbach et al., 2010)

    correlation_measure = ConnectivityMeasure(kind='correlation')
    correlation_matrix = correlation_measure.fit_transform([aal_array])[0]

    threshold = 0.35  
    twenties = 23 
 
    thresholded_matrix = np.where(correlation_matrix >= threshold, 1, 0)
    np.fill_diagonal(thresholded_matrix, 0)
    graph = nx.from_numpy_array(thresholded_matrix)

    label = int(file_name.split('-')[1])

    # degree model
    degree_ = nx.degree(graph)
    degree = dict(degree_).values()
    
    # Betweenness Centrality
    betweenness = nx.betweenness_centrality(graph).values()

    # Clustering Coefficient
    clustering = nx.clustering(graph).values()
    
    # Closeness Centrality
    closeness = nx.closeness_centrality(graph).values()

    # Sort values
    sorted_degree = sorted(degree, reverse=True)
    sorted_betweenness = sorted(betweenness, reverse=True)
    sorted_clustering = sorted(clustering)
    sorted_closeness = sorted(closeness, reverse=True)

    num_values_to_set = 23
    result_degree = [1 if value in sorted_degree[:num_values_to_set] else 0 for value in degree]
    result_betweenness = [1 if value in sorted_betweenness[:num_values_to_set] else 0 for value in betweenness]
    result_clustering = [1 if value in sorted_clustering[:num_values_to_set] else 0 for value in clustering]
    result_closeness = [1 if value in sorted_closeness[:num_values_to_set] else 0 for value in closeness]

    final_result = [result_degree[i] + result_betweenness[i] + result_clustering[i] + result_closeness[i] for i in range(len(degree))]

    if j == 0:
        features_array = final_result
    else:
        features_array = np.vstack((features_array, final_result))

    labels.append(label)
    if j % 100 == 0:
        print(f'----{j} Done)  

0
100
200
300
400
500
600
700
800
900
1000
1100
1200


In [7]:
import numpy as np
a=np.array([1, 3, 5])
b = [3, 5, 6]
np.vstack((a, a))

array([[1, 3, 5],
       [1, 3, 5]])

In [15]:
X = features_array
y = np.array(labels)

In [16]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier  # Import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, auc, roc_curve

#fpath = 'path/for/feature/file/(csv)'
#lpath = 'path/for/label/file/(csv)'
seed = 81 # random seed
no_folds = 10 # number of folds in out_loop
no_nested_folds = 10 # number of folds in nested_loop

skf = StratifiedKFold(n_splits=no_folds, shuffle=True, random_state=101)
nested_skf = StratifiedKFold(n_splits=no_nested_folds, shuffle=True)
param_grid = {'n_estimators': [50, 100, 200, 300], 'max_depth': [None, 10, 20, 30]}  # Modify the parameter grid for RandomForestClassifier
eval_metrics = np.zeros((skf.n_splits, 4))
print('Loading data ...')
# features = np.loadtxt(fpath, delimiter=',')
# labels = np.loadtxt(lpath, dtype='int32')

features = X
labels = y

print(f'Finished')
print(f'{np.sum(labels == 1)} MDD & {np.sum(labels == 2)} HC')

# ROC plotting preparation
TPR, AUC = [], []
mean_fpr = np.linspace(0, 1, 100)

for n_cv, (train_ind, test_ind) in enumerate(skf.split(features, labels)):
    print('Processing the No.%i cross-validation in %i-fold CV' % (n_cv + 1, skf.n_splits))
    x_train, y_train = features[train_ind, ], labels[train_ind, ]
    x_test, y_test = features[test_ind, ], labels[test_ind, ]

    # Training
    init_clf = RandomForestClassifier()
    grid = GridSearchCV(init_clf, param_grid, cv=nested_skf, scoring='accuracy', n_jobs=5)
    grid.fit(x_train, y_train)
    print('----The best parameters: n_estimators=%d, max_depth=%s with accuracy of %f' % (
        grid.best_params_['n_estimators'], grid.best_params_['max_depth'], grid.best_score_))

    clf = RandomForestClassifier(n_estimators=grid.best_params_['n_estimators'],
                                 max_depth=grid.best_params_['max_depth'])
    clf.fit(x_train, y_train)
    y_predict = clf.predict(x_test)
    y_proba = clf.predict_proba(x_test)

    # ROC information for each fold
    cv_fpr, cv_tpr, cv_thresholds = roc_curve(y_test, y_proba[:, 1], pos_label=2)
    cv_auc = auc(cv_fpr, cv_tpr)
    interp_tpr = np.interp(mean_fpr, cv_fpr, cv_tpr)
    interp_tpr[0] = 0.0
    TPR.append(interp_tpr)
    AUC.append(cv_auc)

    # Evaluation
    tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
    cv_accuracy = (tn + tp) / (tn + fp + fn + tp)
    cv_sensitivity = tp / (tp + fn)
    cv_specificity = tn / (tn + fp)
    eval_metrics[n_cv, 0] = cv_accuracy
    eval_metrics[n_cv, 1] = cv_sensitivity
    eval_metrics[n_cv, 2] = cv_specificity
    eval_metrics[n_cv, 3] = cv_auc

# reporting model evaluation measures
df = pd.DataFrame(eval_metrics)
df.columns = ['ACC', 'SEN', 'SPE', 'AUC']
df.index = ['CV_' + str(i + 1) for i in range(skf.n_splits)]
print(df)
print('\nAverage Accuracy: %.4f' % (eval_metrics[:, 0].mean()))
print('Average Sensitivity: %.4f' % (eval_metrics[:, 1].mean()))
print('Average Specificity: %.4f' % (eval_metrics[:, 2].mean()))
print('Average area under ROC curve: %.4f' % (eval_metrics[:, 3].mean()))

# saving ROC plotting information
# mean_tpr = np.mean(TPR, axis=0)
# mean_tpr[-1] = 1.0
# mean_auc = auc(mean_fpr, mean_tpr)
# np.savez(fpath + '/../ROC_MTR.npz', tpr=mean_tpr, fpr=mean_fpr, auc=mean_auc)

Loading data ...
Finished
717 MDD & 583 HC
Processing the No.1 cross-validation in 10-fold CV
----The best parameters: n_estimators=300, max_depth=30 with accuracy of 0.575214
Processing the No.2 cross-validation in 10-fold CV
----The best parameters: n_estimators=300, max_depth=30 with accuracy of 0.560684
Processing the No.3 cross-validation in 10-fold CV
----The best parameters: n_estimators=300, max_depth=None with accuracy of 0.569231
Processing the No.4 cross-validation in 10-fold CV
----The best parameters: n_estimators=300, max_depth=20 with accuracy of 0.560684
Processing the No.5 cross-validation in 10-fold CV
----The best parameters: n_estimators=200, max_depth=30 with accuracy of 0.570085
Processing the No.6 cross-validation in 10-fold CV
----The best parameters: n_estimators=300, max_depth=20 with accuracy of 0.584615
Processing the No.7 cross-validation in 10-fold CV
----The best parameters: n_estimators=300, max_depth=None with accuracy of 0.563248
Processing the No.8 cr